<a href="https://colab.research.google.com/github/rudycav/RedditScraper/blob/edit/Project/r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install praw
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import praw
from praw.models import MoreComments
import pandas as pd
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from nltk.tokenize import word_tokenize, sent_tokenize, RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from collections import Counter
import re
import requests

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [215]:
reddit = praw.Reddit(
    client_id="BE3T0CMpNrtqLT97yL6XiA",
    client_secret = "gJnvUXTK0gCnj3UMGSUtw0JW35tN1A",
    password = "Hannah00!!@",
    user_agent = "spidey by u/rudyboycav",
    username = "rudyboycav",
    check_for_async=False
)

url='https://web-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
for start in range(1, 100, 1000):
    params = {
        'start': start,
        'limit': 200,
            }
    r = requests.get(url, params=params)
    data = r.json()

ticker = [(crypto['symbol'].lower()) for crypto in (data['data'])]
slug = [(crypto['name'].lower()) for crypto in (data['data'])]



def reddit_scraper(reddit, subreddit_page, limit):
  df = pd.DataFrame()
  subreddit = reddit.subreddit(subreddit_page)
  hot_topic = subreddit.hot(limit=limit)

  for x in hot_topic: 
    c = x.comments.list()
    for y in c:
      if isinstance(y, MoreComments):
        continue
      df = df.append({#'title': x.title,
                      #'author': y.author,
                      'comments': y.body,
                      #'upvote_score': y.score
                      },                                          
                      ignore_index=True) 

  return df

wall = reddit_scraper(reddit, 'CryptoCurrency', 9)

In [216]:
tokenizer = RegexpTokenizer(r'\w+')
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()

def text_cleaner(df):
    cleaned_text = []
    for text in df:
        tokenize_words = tokenizer.tokenize(text)      
        stop_word_filter = [token.lower() for token in tokenize_words if token.lower() not in stop_words and not token.isdigit()]
        lemmitize_words = ([lemmatizer.lemmatize(token) for token in stop_word_filter])
        cleaned_text.extend(lemmitize_words)
    
    return cleaned_text

comments = text_cleaner(wall.comments)

sia = SIA()

ps = []
for comment in comments:
    score = sia.polarity_scores(comment)
    score['replies'] = comment
    ps.append(score)

df2 = pd.DataFrame.from_records(ps)

df2['label'] = 0
df2.loc[df2['compound'] > 0.1, 'label'] = 1
df2.loc[df2['compound'] < -0.1, 'label'] = -1


In [217]:
'''
sia = SIA()
ticker_mentions = []
name_mentions = []
ff = []

#ticker_mentions = [pol_score = sia.polarity_scores(y) for y in wall.comments for x in ticker if x in y]


for x in ticker:
  for y in wall.comments:
    if x in y:
      if y not in ticker_mentions:
        ticker_mentions.append(y)
      else:
        ticker_mentions = ticker_mentions
'''       
          




'\nsia = SIA()\nticker_mentions = []\nname_mentions = []\nff = []\n\n#ticker_mentions = [pol_score = sia.polarity_scores(y) for y in wall.comments for x in ticker if x in y]\n\n\nfor x in ticker:\n  for y in wall.comments:\n    if x in y:\n      if y not in ticker_mentions:\n        ticker_mentions.append(y)\n      else:\n        ticker_mentions = ticker_mentions\n'

In [218]:
for x in wall.comments:
  print(x)


Bullish factors for 2023:

- we already dropped so much, historically we never have 2 red years in a row
- inflation is coming down, meaning rate hikes will stop at some point, and maybe rate cuts will happen in the second half of the year
- markets are forward looking, meaning that most of the bad stuff is already priced in
- Halving getting closer
- Adoption is still high and increasing
- BTC in exchanges are at a low level and still dropping (supply shock)
- those who had losses on their portfolio likely realized these in 2022 ( less selling pressure in 2023)
- Those who haven’t already sold are HODLing
I just need to be filled with hopium for this month so I can go all in soon! Judging from the only two comments here, it seems all the bulls are gone. They do make great points too. Just want everyone to know you cant predict the bottom; have a nice year everyone!

&#x200B;

I pretty much believe we are gonna go up, maybe not now but in the second half of 2023 at least. Feeling ready

In [219]:
pos_lines = list(df2[df2.label == 1].replies)

pos_tokens = text_cleaner(pos_lines)
#pos_freq = nltk.FreqDist(pos_tokens)

#pos_freq.most_common(20)
pos_tokens

['great',
 'nice',
 'pretty',
 'feeling',
 'ready',
 'interest',
 'increased',
 'good',
 'huge',
 'gain',
 'honestly',
 'best',
 'accept',
 'great',
 'good',
 'positive',
 'cool',
 'improve',
 'speculative',
 'growth',
 'hope',
 'better',
 'truly',
 'created',
 'ready',
 'easy',
 'holiday',
 'credit',
 'confidence',
 'like',
 'surprise',
 'like',
 'like',
 'created',
 'play',
 'safe',
 'positive',
 'diamond',
 'hand',
 'ready',
 'like',
 'good',
 'bonus',
 'interest',
 'positive',
 'hope',
 'truth',
 'curious',
 'optimistic',
 'hope',
 'appreciation',
 'optimism',
 'increase',
 'stable',
 'stable',
 'appreciating',
 'appreciation',
 'yes',
 'please',
 'yes',
 'please',
 'thanks',
 'support',
 'support',
 'growth',
 'celebrate',
 'rich',
 'huge',
 'gain',
 'honest',
 'like',
 'increase',
 'friend',
 'like',
 'increased',
 'benefit',
 'like',
 'free',
 'like',
 'like',
 'good',
 'yes',
 'increased',
 'increase',
 'bright',
 'like',
 'lol',
 'truly',
 'perfect',
 'best',
 'great',
 'hope'

In [220]:
neg_lines = list(df2[df2.label == -1].replies)

neg_tokens = text_cleaner(neg_lines)
#pos_freq = nltk.FreqDist(pos_tokens)

#pos_freq.most_common(20)
neg_tokens

['stop',
 'cut',
 'bad',
 'low',
 'shock',
 'loss',
 'pressure',
 'lower',
 'low',
 'low',
 'war',
 'severely',
 'falling',
 'lower',
 'low',
 'low',
 'lower',
 'drop',
 'unemployment',
 'unemployment',
 'low',
 'unemployment',
 'worry',
 'disappear',
 'recession',
 'debt',
 'low',
 'worried',
 'recession',
 'recession',
 'doom',
 'negative',
 'recession',
 'recession',
 'lower',
 'disheartened',
 'bother',
 'anti',
 'pessimistic',
 'stop',
 'seriously',
 'lower',
 'drop',
 'ignore',
 'low',
 'fud',
 'pay',
 'pay',
 'lost',
 'crisis',
 'bad',
 'war',
 'crisis',
 'crisis',
 'crisis',
 'uncertainty',
 'drop',
 'losing',
 'losing',
 'lose',
 'depressed',
 'drop',
 'restriction',
 'cheating',
 'dead',
 'unfortunately',
 'fud',
 'low',
 'problem',
 'stop',
 'crap',
 'risky',
 'risk',
 'hurt',
 'crisis',
 'crisis',
 'unemployment',
 'pay',
 'pay',
 'risky',
 'risk',
 'dead',
 'problem',
 'loss',
 'pay',
 'banned',
 'dump',
 'pay',
 'evil',
 'crisis',
 'blaming',
 'stop',
 'lost',
 'scam',
 '

In [221]:
neu_lines = list(df2[df2.label == 0].replies)

neu_tokens = text_cleaner(neu_lines)
#pos_freq = nltk.FreqDist(pos_tokens)

#pos_freq.most_common(20)
neu_tokens

['bullish',
 'factor',
 'already',
 'dropped',
 'much',
 'historically',
 'never',
 'red',
 'year',
 'row',
 'inflation',
 'coming',
 'meaning',
 'rate',
 'hike',
 'point',
 'maybe',
 'rate',
 'happen',
 'second',
 'half',
 'year',
 'market',
 'forward',
 'looking',
 'meaning',
 'stuff',
 'already',
 'priced',
 'halving',
 'getting',
 'closer',
 'adoption',
 'still',
 'high',
 'increasing',
 'btc',
 'exchange',
 'level',
 'still',
 'dropping',
 'supply',
 'portfolio',
 'likely',
 'realized',
 'le',
 'selling',
 'already',
 'sold',
 'hodling',
 'need',
 'filled',
 'hopium',
 'month',
 'go',
 'soon',
 'judging',
 'two',
 'comment',
 'seems',
 'bull',
 'gone',
 'make',
 'point',
 'want',
 'everyone',
 'know',
 'cant',
 'predict',
 'bottom',
 'year',
 'everyone',
 'x200b',
 'much',
 'believe',
 'gonna',
 'go',
 'maybe',
 'second',
 'half',
 'least',
 'inevitably',
 'hit',
 'opinion',
 'crypto',
 'market',
 'primarily',
 'influenced',
 'rate',
 'bitcoin',
 'halving',
 'coincidently',
 'happ

In [222]:
vv = []
dc = []
nn = []

for x in pos_tokens:
  for y in ticker:
    if x == y:
      vv.append(y)

for x in neg_tokens:
  for y in ticker:
    if x == y:
      dc.append(y)

for x in neu_tokens:
  for y in ticker:
    if x == y:
      nn.append(y)

In [223]:
ccc = Counter(dc)
ccc = ccc.most_common()
ccc

[]

In [224]:
ggg = Counter(nn)

gggg =ggg.most_common(5)
gggg

[('people', 202), ('one', 98), ('btc', 36), ('eth', 25), ('sol', 20)]

In [225]:
vfv = Counter(vv)
vfvv = vfv.most_common()
vfvv

[]

In [226]:
for x in name:
  for y in wall.comments:
    if x in y:
      if y not in name_mentions:
          pol_score = sia.polarity_scores(y)
          pol_score['replies'] = y
          name_mentions.append(pol_score)

NameError: ignored

In [ ]:
name_mentions

In [ ]:
li = {}
dd = []

for s in ticker:
  for x in wall.comments:
    if s in x:
      
      if s not in li:
 
        li[s] = 1
      else:
        li[s] += 1

In [ ]:
li

In [ ]:
for x in wall.comments:
  ff = x.split()
  pp = list(set(filter(lambda ff: ff.lower().startswith('$'), ff)))
  output = re.sub(r'\d+', '', str(pp))


  if len(pp) > 0:
    print(output)




In [ ]:
for x in tokey:
  print(x.startswith('$'))

In [ ]:
for x in xx:
  print(x)

In [ ]:
def clean_text(df):
  clean_text = " ".join(str(df).split())
  clean_text = " ".join([w for w in df.split() if w.isalpha()])

  return clean_text

gg = wall.comments.map(clean_text)

In [ ]:
for x in gg:
  print(x)

In [ ]:
def word_tokenizer(df):
  tokens = word_tokenize(str(df))

  return tokens

ggg =word_tokenizer(gg)

In [ ]:
print(ggg)
print(len(ggg))

In [ ]:
stopwords_ = set(stopwords.words("english"))

In [ ]:

def stopwords(df):
  clean_tokens = [t for t in df if not t in stopwords_]
  clean_text = " ".join(clean_tokens)

  return clean_tokens

gggg = stopwords(ggg)

In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize('cats')

In [ ]:
for x in gggg:
  ff = lemmatizer.lemmatize(x)
  print(x)

In [ ]:
def lemmatizer(df):
  return [lemmatizer.lemmatize(word) for word in df]
gggg.map(str(lemmatizer))


In [ ]:
print(gggg)
print(len(gggg))

In [ ]:
non_letter = "".join([x for x in str(cleaned_text) if x not in string.punctuation])
non_letter

In [ ]:
def remove_punctuation(df):

  #non_letter = re.sub(r"https?://\S+", "", str(df))
  non_letter = "".join([x for x in str(df) if x not in string.punctuation]) #removes punctuation
  non_letter = ''.join([i for i in str(df) if not i.isdigit()]) #removes numbers
  #non_letter = demojize(str(df))
  
  return non_letter

cleaned_text = remove_punctuation(cleaned_text)

In [ ]:
cleaned_text

In [ ]:
for x in cleaned_text:
  if x is 'SPY':
    print(x)

In [ ]:
for x in cleaned_text:
  if x.isupper():
    print(x)

In [ ]:
def tokenize(df):
  for words in wall:
    tokenize = word_tokenize(words)
  return tokenize

tokenized_words = tokenize(wall)

In [ ]:
stop_words = set(stopwords.words("english"))

In [ ]:
def stopwords(df):
  #stop_words = set(stopwords.words("english"))
  filtered_words = [word for word in df if word.casefold() not in stop_words]
  return filtered_words

filtered_stopwords = stopwords(tokenized_words)
filtered_stopwords

In [ ]:
def stemming(df):
  stemmed_list = []
  snow_stemmer = SnowballStemmer(language='english')
  
  for words in filtered_stopwords:
    stemmer = snow_stemmer.stem(words)
    stemmed_list.append(stemmer)
  return stemmed_list

stemmed_words = stemming(filtered_stopwords)
stemmed_words

In [ ]:
subreddit = await reddit.subreddit("wallstreetbets")
async for x in subreddit.hot(limit=3):
    print('THREAD: {}'.format(x.title))
    print('USER: {}'.format(x.author))

    print(x.comments)
    

